In [ ]:
# Create Stash app client

import pandas as pd
import dotenv

from libraries.client_stashapp import get_stashapp_client

dotenv.load_dotenv()

stash = get_stashapp_client()

In [ ]:
tpdb_markers = stash.find_tag("Markers: TPDB")
tpdb_markers

In [ ]:
ai_tagged = stash.find_tag("AI_Tagged")
ai_tagged

In [ ]:
marker_source_tpdb = stash.find_tag("Marker Source: TPDB")
marker_source_tpdb

In [ ]:
scenes_with_only_tpdb_markers = stash.find_scenes({
    "tags": {
        "depth": -1,
        "value": [tpdb_markers["id"]],
        "modifier": "INCLUDES_ALL",
        "excludes": [ai_tagged["id"]],
    }
}, fragment="id title code details scene_markers { id title primary_tag { id name } tags { id name } }")

In [ ]:
df_scenes_with_only_tpdb_markers = pd.DataFrame(scenes_with_only_tpdb_markers)
# Add a new column for the number of markers
df_scenes_with_only_tpdb_markers['num_markers'] = df_scenes_with_only_tpdb_markers['scene_markers'].apply(lambda x: len(x) if isinstance(x, list) else 0)

df_scenes_with_only_tpdb_markers = df_scenes_with_only_tpdb_markers[df_scenes_with_only_tpdb_markers['num_markers'] > 1]

In [ ]:
for index, row in df_scenes_with_only_tpdb_markers.iterrows():
    # Check if all scene_markers have empty tags or only contain Marker Source: TPDB tag
    markers_empty = []
    markers_only_tpdb = []
    markers_invalid = []

    for marker in row['scene_markers']:
        if len(marker['tags']) == 0:
            markers_empty.append(marker)
        elif len(marker['tags']) == 1 and marker['tags'][0]['name'] == "Marker Source: TPDB":
            markers_only_tpdb.append(marker)
        else:
            markers_invalid.append(marker)

    all_markers_valid = len(markers_invalid) == 0

    print(f"All markers have empty tags or only Marker Source: TPDB tag: {all_markers_valid}")
    print(f"Markers with empty tags: {len(markers_empty)}")
    print(f"Markers with only Marker Source: TPDB tag: {len(markers_only_tpdb)}")

    if markers_invalid:
        print("\nMarkers with invalid tags:")
        for marker in markers_invalid:
            print(f"Marker ID: {marker['id']}, Title: {marker['title']}, Tags: {[tag['name'] for tag in marker['tags']]}")
    else:
        print("\nNo markers found with invalid tags.")

    # Update markers with empty tags to include Marker Source: TPDB tag
    for marker in markers_empty:
        stash.update_scene_marker({
            "id": marker["id"],
            "tag_ids": [marker_source_tpdb["id"]],
        })

    print(f"Updated {len(markers_empty)} markers with empty tags to include Marker Source: TPDB tag.")


In [ ]:
for marker in scene_markers:
    stash.update_scene_marker({
        "id": marker["id"],
        "tag_ids": [marker_source_tpdb["id"]],
    })

In [ ]:
df_scenes_with_only_tpdb_markers = pd.DataFrame(scenes_with_only_tpdb_markers)